## Import libraries


In [133]:
from sklearn.svm import SVR
import pandas as pd
import os
from imblearn.pipeline import Pipeline as ImbalancedPipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer

%pip install pandas 
%pip install matplotlib
%pip install seaborn
%pip install plotly


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [134]:
current_path = os.getcwd()
filepath = "/Users/laumeijin/Desktop/datathon/" #pls change accordingly

## Read in data


In [135]:
train_dom_data = pd.read_csv(filepath + "data/Xdom_train_data.csv")
test_dom_data = pd.read_csv(filepath + "data/Xdom_test_data.csv")
train_global_data = pd.read_csv(filepath + "data/Xglobal_train_data.csv")
test_global_data = pd.read_csv(filepath + "data/Xglobal_test_data.csv")

## For Domestic


In [136]:
# split train_dom_data into X_train and y_train for domestic

X_train_dom = train_dom_data.iloc[:, 0:83]
y_train_dom = train_dom_data.iloc[:, 84] 

# split test_dom_data into X_test_dom and y_test_dom for domestic

X_test_dom = test_dom_data.iloc[:, 0:83]
y_test_dom = test_dom_data.iloc[:, 84] 

### SVR linear kernel


In [137]:
svr_linear = ImbalancedPipeline(steps=[
    ('classifier', SVR(kernel='linear'))
])

kf = StratifiedKFold(n_splits=5, shuffle=False)

In [138]:
scoring_table = pd.DataFrame(columns=['Model', 'MSE', 'MAE', 'R2', 'Adjusted_R2'])
r2_scorer = make_scorer(r2_score)

# Add regression scoring functions
def model_evaluation_linear(regressor, param_grid, scoring_table, model_name, X_train_encoded, y_train, X_test_encoded, y_test):
    # training the model
    grid_search = GridSearchCV(regressor, param_grid, cv=kf, scoring=r2_scorer, n_jobs=-1)
    grid_search.fit(X_train_encoded, y_train)

    # fit the model with the test set using the best parameters
    best_model = grid_search.best_estimator_
    y_train_pred = best_model.predict(X_train_encoded)
    y_pred = best_model.predict(X_test_encoded)

    # evaluating the model using the test set
    # calculate mean squared error
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_pred)

    # calculate mean absolute error
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_pred)

    # calculate R-squared
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_pred)

    # calculate adjusted R-squared
    n_train, k_train = X_train_encoded.shape[0], X_train_encoded.shape[1]
    n_test, k_test = X_test_encoded.shape[0], X_test_encoded.shape[1]

    adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - k_train - 1))
    adj_r2_test = 1 - ((1 - r2_test) * (n_test - 1) / (n_test - k_test - 1))

    # record the performance of the model
    scoring_table = pd.concat([scoring_table, pd.DataFrame({
        'Model': model_name + '_train',
        'Best_iteration': grid_search.best_params_['classifier__max_iter'], 
        'Best_C': grid_search.best_params_['classifier__C'],
        'MSE': mse_train,
        'MAE': mae_train,
        'R2': r2_train,
        'Adjusted_R2': adj_r2_train,
    }, index=[0])])

    scoring_table = pd.concat([scoring_table, pd.DataFrame({
        'Model': model_name + '_test',
        'Best_iteration': grid_search.best_params_['classifier__max_iter'], 
        'Best_C': grid_search.best_params_['classifier__C'],
        'MSE': mse_test,
        'MAE': mae_test,
        'R2': r2_test,
        'Adjusted_R2': adj_r2_test,
    }, index=[0])])

    return scoring_table

In [139]:
param_grid_1 = {
    'classifier__max_iter': [10, 100, 1000, 1500], 
    'classifier__C': [0.01, 0.1, 1.0, 10.0, 100.0]  
}

scoring_table = model_evaluation_linear(svr_linear, param_grid_1, scoring_table, 'svr_linear_dom', X_train_dom, y_train_dom, X_test_dom, y_test_dom)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Librar

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C
0,svr_linear_dom_train,9.082048e+19,7.934777e+08,-0.001743,-0.008335,1000.0,100.0
0,svr_linear_dom_test,5.198952e+19,7.035097e+08,-0.001841,-0.028743,1000.0,100.0


In [140]:
param_grid_1 = {
    'classifier__max_iter': [700, 800, 1000, 1100, 1200], 
    'classifier__C': [80.0, 90.0, 100.0, 110.0, 120.0]  
}

scoring_table = model_evaluation_linear(svr_linear, param_grid_1, scoring_table, 'svr_linear_dom_2', X_train_dom, y_train_dom, X_test_dom, y_test_dom)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=700).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=700).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=700).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Lib

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C
0,svr_linear_dom_train,9.082048e+19,7.934777e+08,-0.001743,-0.008335,1000.0,100.0
0,svr_linear_dom_test,5.198952e+19,7.035097e+08,-0.001841,-0.028743,1000.0,100.0
0,svr_linear_dom_2_train,9.070074e+19,9.567820e+08,-0.000422,-0.007006,700.0,120.0
0,svr_linear_dom_2_test,5.190548e+19,8.670578e+08,-0.000222,-0.027080,700.0,120.0


In [141]:
param_grid_1 = {
    'classifier__max_iter': [500, 600, 650, 700, 750], 
    'classifier__C': [120, 130, 140, 150]  
}

scoring_table = model_evaluation_linear(svr_linear, param_grid_1, scoring_table, 'svr_linear_dom_3', X_train_dom, y_train_dom, X_test_dom, y_test_dom)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Lib

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C
0,svr_linear_dom_train,9.082048e+19,7.934777e+08,-0.001743,-0.008335,1000.0,100.0
0,svr_linear_dom_test,5.198952e+19,7.035097e+08,-0.001841,-0.028743,1000.0,100.0
0,svr_linear_dom_2_train,9.070074e+19,9.567820e+08,-0.000422,-0.007006,700.0,120.0
0,svr_linear_dom_2_test,5.190548e+19,8.670578e+08,-0.000222,-0.027080,700.0,120.0
0,svr_linear_dom_3_train,9.067100e+19,1.204612e+09,-0.000094,-0.006675,500.0,150.0
0,svr_linear_dom_3_test,5.192669e+19,1.115522e+09,-0.000630,-0.027499,500.0,150.0


MSE for test set did not improve further so we will stop our hyperparameter tuning. Best result for linear kernel would be: best_iteration = 700 and best_c = 120.


### SVR polynomial kernel


In [142]:
svr_poly = ImbalancedPipeline(steps=[
    ('classifier', SVR(kernel='poly'))
])

In [143]:
scoring_table = pd.DataFrame(columns=['Model', 'MSE', 'MAE', 'R2', 'Adjusted_R2'])
r2_scorer = make_scorer(r2_score)

# Add regression scoring functions
def model_evaluation_poly(regressor, param_grid, scoring_table, model_name, X_train_encoded, y_train, X_test_encoded, y_test):
    # training the model
    grid_search = GridSearchCV(regressor, param_grid, cv=kf, scoring=r2_scorer, n_jobs=-1)
    grid_search.fit(X_train_encoded, y_train)

    # fit the model with the test set using the best parameters
    best_model = grid_search.best_estimator_
    y_train_pred = best_model.predict(X_train_encoded)
    y_pred = best_model.predict(X_test_encoded)

    # evaluating the model using the test set
    # calculate mean squared error
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_pred)

    # calculate mean absolute error
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_pred)

    # calculate R-squared
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_pred)

    # calculate adjusted R-squared
    n_train, k_train = X_train_encoded.shape[0], X_train_encoded.shape[1]
    n_test, k_test = X_test_encoded.shape[0], X_test_encoded.shape[1]

    adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - k_train - 1))
    adj_r2_test = 1 - ((1 - r2_test) * (n_test - 1) / (n_test - k_test - 1))

    # record the performance of the model
    scoring_table = pd.concat([scoring_table, pd.DataFrame({
        'Model': model_name + '_train',
        'Best_iteration': grid_search.best_params_['classifier__max_iter'], 
        'Best_C': grid_search.best_params_['classifier__C'],
        'Best_degree': grid_search.best_params_['classifier__degree'],
        'Best_gamma': grid_search.best_params_['classifier__gamma'],
        'MSE': mse_train,
        'MAE': mae_train,
        'R2': r2_train,
        'Adjusted_R2': adj_r2_train,
    }, index=[0])])

    scoring_table = pd.concat([scoring_table, pd.DataFrame({
        'Model': model_name + '_test',
        'Best_iteration': grid_search.best_params_['classifier__max_iter'], 
        'Best_C': grid_search.best_params_['classifier__C'],
        'Best_degree': grid_search.best_params_['classifier__degree'],
        'Best_gamma': grid_search.best_params_['classifier__gamma'],
        'MSE': mse_test,
        'MAE': mae_test,
        'R2': r2_test,
        'Adjusted_R2': adj_r2_test,
    }, index=[0])])

    return scoring_table

In [144]:
param_grid_1 = {
    'classifier__max_iter': [10, 100, 1000, 2000], 
    'classifier__C': [0.01, 0.1, 1.0, 10.0, 100.0],  
    'classifier__degree': [3, 4, 5],
    'classifier__gamma': [0.01, 0.1, 1.0, 10.0]
}

scoring_table = model_evaluation_poly(svr_poly, param_grid_1, scoring_table, 'svr_poly_dom', X_train_dom, y_train_dom, X_test_dom, y_test_dom)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Librar

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C,Best_degree,Best_gamma
0,svr_poly_dom_train,9.081848e+19,7.954349e+08,-0.001721,-0.008313,1000.0,1.0,3.0,1.0
0,svr_poly_dom_test,5.198796e+19,7.054571e+08,-0.001811,-0.028712,1000.0,1.0,3.0,1.0


In [145]:
param_grid_1 = {
    'classifier__max_iter': [800, 900, 1000, 1100, 1200], 
    'classifier__C': [0.8, 0.9, 1.0, 1.1, 1.2],  
    'classifier__degree': [3, 4, 5],
    'classifier__gamma': [0.8, 0.9, 1.0, 1.1, 1.2]
}

scoring_table = model_evaluation_poly(svr_poly, param_grid_1, scoring_table, 'svr_poly_dom_2', X_train_dom, y_train_dom, X_test_dom, y_test_dom)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Lib

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C,Best_degree,Best_gamma
0,svr_poly_dom_train,9.081848e+19,7.954349e+08,-0.001721,-0.008313,1000.0,1.0,3.0,1.0
0,svr_poly_dom_test,5.198796e+19,7.054571e+08,-0.001811,-0.028712,1000.0,1.0,3.0,1.0
0,svr_poly_dom_2_train,9.073419e+19,8.971298e+08,-0.000791,-0.007377,800.0,1.2,3.0,1.0
0,svr_poly_dom_2_test,5.192617e+19,8.070368e+08,-0.000620,-0.027489,800.0,1.2,3.0,1.0


In [146]:
param_grid_1 = {
    'classifier__max_iter': [700, 750, 800, 850], 
    'classifier__C': [1.2, 1.3, 1.4, 1.5],  
    'classifier__degree': [3],
    'classifier__gamma': [0.8, 0.9, 1.0, 1.1, 1.2]
}

scoring_table = model_evaluation_poly(svr_poly, param_grid_1, scoring_table, 'svr_poly_dom_3', X_train_dom, y_train_dom, X_test_dom, y_test_dom)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=700).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=700).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=700).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Lib

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C,Best_degree,Best_gamma
0,svr_poly_dom_train,9.081848e+19,7.954349e+08,-0.001721,-0.008313,1000.0,1.0,3.0,1.0
0,svr_poly_dom_test,5.198796e+19,7.054571e+08,-0.001811,-0.028712,1000.0,1.0,3.0,1.0
0,svr_poly_dom_2_train,9.073419e+19,8.971298e+08,-0.000791,-0.007377,800.0,1.2,3.0,1.0
0,svr_poly_dom_2_test,5.192617e+19,8.070368e+08,-0.000620,-0.027489,800.0,1.2,3.0,1.0
0,svr_poly_dom_3_train,9.070041e+19,9.574934e+08,-0.000419,-0.007002,700.0,1.5,3.0,1.0
0,svr_poly_dom_3_test,5.190530e+19,8.677758e+08,-0.000218,-0.027076,700.0,1.5,3.0,1.0


In [147]:
param_grid_1 = {
    'classifier__max_iter': [600, 650, 700, 750], 
    'classifier__C': [1.5, 1.6, 1.7, 1.8],  
    'classifier__degree': [3],
    'classifier__gamma': [0.8, 0.9, 1.0, 1.1, 1.2]
}

scoring_table = model_evaluation_poly(svr_poly, param_grid_1, scoring_table, 'svr_poly_dom_4', X_train_dom, y_train_dom, X_test_dom, y_test_dom)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=600).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=600).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=600).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Lib

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C,Best_degree,Best_gamma
0,svr_poly_dom_train,9.081848e+19,7.954349e+08,-0.001721,-0.008313,1000.0,1.0,3.0,1.0
0,svr_poly_dom_test,5.198796e+19,7.054571e+08,-0.001811,-0.028712,1000.0,1.0,3.0,1.0
0,svr_poly_dom_2_train,9.073419e+19,8.971298e+08,-0.000791,-0.007377,800.0,1.2,3.0,1.0
0,svr_poly_dom_2_test,5.192617e+19,8.070368e+08,-0.000620,-0.027489,800.0,1.2,3.0,1.0
0,svr_poly_dom_3_train,9.070041e+19,9.574934e+08,-0.000419,-0.007002,700.0,1.5,3.0,1.0
0,svr_poly_dom_3_test,5.190530e+19,8.677758e+08,-0.000218,-0.027076,700.0,1.5,3.0,1.0
0,svr_poly_dom_4_train,9.066561e+19,1.075306e+09,-0.000035,-0.006616,600.0,1.8,3.0,1.0
0,svr_poly_dom_4_test,5.189503e+19,9.863483e+08,-0.000020,-0.026873,600.0,1.8,3.0,1.0


Althoug the MSE still remains rather large, the polynomial kernel did slightly better than the linear kernel. This suggests that there is a non-linear relationship between the input features and the sales in our data. Best result: best_iteration = 600, best_c = 1.8, best_degree = 3, best_gamma = 1.


## For Global


### SVR linear kernel


In [148]:
# split train_data into X_train and y_train for global

X_train_global = train_global_data.iloc[:, 0:83]
y_train_global = train_global_data.iloc[:, 84] 

# split test_global_data into X_test_globl and y_test_global for global

X_test_global = test_global_data.iloc[:, 0:83]
y_test_global = test_global_data.iloc[:, 84]

In [149]:
scoring_table = pd.DataFrame(columns=['Model', 'MSE', 'MAE', 'R2', 'Adjusted_R2'])
r2_scorer = make_scorer(r2_score)

# Add regression scoring functions
def model_evaluation_linear(regressor, param_grid, scoring_table, model_name, X_train_encoded, y_train, X_test_encoded, y_test):
    # training the model
    grid_search = GridSearchCV(regressor, param_grid, cv=kf, scoring=r2_scorer, n_jobs=-1)
    grid_search.fit(X_train_encoded, y_train)

    # fit the model with the test set using the best parameters
    best_model = grid_search.best_estimator_
    y_train_pred = best_model.predict(X_train_encoded)
    y_pred = best_model.predict(X_test_encoded)

    # evaluating the model using the test set
    # calculate mean squared error
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_pred)

    # calculate mean absolute error
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_pred)

    # calculate R-squared
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_pred)

    # calculate adjusted R-squared
    n_train, k_train = X_train_encoded.shape[0], X_train_encoded.shape[1]
    n_test, k_test = X_test_encoded.shape[0], X_test_encoded.shape[1]

    adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - k_train - 1))
    adj_r2_test = 1 - ((1 - r2_test) * (n_test - 1) / (n_test - k_test - 1))

    # record the performance of the model
    scoring_table = pd.concat([scoring_table, pd.DataFrame({
        'Model': model_name + '_train',
        'Best_iteration': grid_search.best_params_['classifier__max_iter'], 
        'Best_C': grid_search.best_params_['classifier__C'],
        'MSE': mse_train,
        'MAE': mae_train,
        'R2': r2_train,
        'Adjusted_R2': adj_r2_train,
    }, index=[0])])

    scoring_table = pd.concat([scoring_table, pd.DataFrame({
        'Model': model_name + '_test',
        'Best_iteration': grid_search.best_params_['classifier__max_iter'], 
        'Best_C': grid_search.best_params_['classifier__C'],
        'MSE': mse_test,
        'MAE': mae_test,
        'R2': r2_test,
        'Adjusted_R2': adj_r2_test,
    }, index=[0])])

    return scoring_table

In [150]:
param_grid_1 = {
    'classifier__max_iter': [10, 100, 1000, 1500], 
    'classifier__C': [0.01, 0.1, 1.0, 10.0, 100.0]  
}

scoring_table = model_evaluation_linear(svr_linear, param_grid_1, scoring_table, 'svr_linear_global', X_train_global, y_train_global, X_test_global, y_test_global)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Librar

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C
0,svr_linear_global_train,7.204950e+20,9.770088e+09,-0.001272,-0.007860,1000.0,100.0
0,svr_linear_global_test,7.924180e+20,1.006989e+10,-0.000383,-0.027245,1000.0,100.0


In [151]:
param_grid_1 = {
    'classifier__max_iter': [800, 900, 1000, 1100], 
    'classifier__C': [50.0, 100.0, 150.0, 200.0]  
}

scoring_table = model_evaluation_linear(svr_linear, param_grid_1, scoring_table, 'svr_linear_global_2', X_train_global, y_train_global, X_test_global, y_test_global)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Lib

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C
0,svr_linear_global_train,7.204950e+20,9.770088e+09,-0.001272,-0.007860,1000.0,100.0
0,svr_linear_global_test,7.924180e+20,1.006989e+10,-0.000383,-0.027245,1000.0,100.0
0,svr_linear_global_2_train,7.262332e+20,1.105940e+10,-0.009246,-0.015887,900.0,50.0
0,svr_linear_global_2_test,7.968388e+20,1.133883e+10,-0.005964,-0.032976,900.0,50.0


MSE performed worse with hyperparameter tuning. MSE for global sales forecast is larger than MSE for domestic sales forecast.


### SVR polynomial kernel


In [ ]:
svr_poly = ImbalancedPipeline(steps=[
    ('classifier', SVR(kernel='poly'))
])

In [152]:
scoring_table = pd.DataFrame(columns=['Model', 'MSE', 'MAE', 'R2', 'Adjusted_R2'])
r2_scorer = make_scorer(r2_score)

# Add regression scoring functions
def model_evaluation_poly(regressor, param_grid, scoring_table, model_name, X_train_encoded, y_train, X_test_encoded, y_test):
    # training the model
    grid_search = GridSearchCV(regressor, param_grid, cv=kf, scoring=r2_scorer, n_jobs=-1)
    grid_search.fit(X_train_encoded, y_train)

    # fit the model with the test set using the best parameters
    best_model = grid_search.best_estimator_
    y_train_pred = best_model.predict(X_train_encoded)
    y_pred = best_model.predict(X_test_encoded)

    # evaluating the model using the test set
    # calculate mean squared error
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_pred)

    # calculate mean absolute error
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_pred)

    # calculate R-squared
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_pred)

    # calculate adjusted R-squared
    n_train, k_train = X_train_encoded.shape[0], X_train_encoded.shape[1]
    n_test, k_test = X_test_encoded.shape[0], X_test_encoded.shape[1]

    adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - k_train - 1))
    adj_r2_test = 1 - ((1 - r2_test) * (n_test - 1) / (n_test - k_test - 1))

    # record the performance of the model
    scoring_table = pd.concat([scoring_table, pd.DataFrame({
        'Model': model_name + '_train',
        'Best_iteration': grid_search.best_params_['classifier__max_iter'], 
        'Best_C': grid_search.best_params_['classifier__C'],
        'Best_degree': grid_search.best_params_['classifier__degree'],
        'Best_gamma': grid_search.best_params_['classifier__gamma'],
        'MSE': mse_train,
        'MAE': mae_train,
        'R2': r2_train,
        'Adjusted_R2': adj_r2_train,
    }, index=[0])])

    scoring_table = pd.concat([scoring_table, pd.DataFrame({
        'Model': model_name + '_test',
        'Best_iteration': grid_search.best_params_['classifier__max_iter'], 
        'Best_C': grid_search.best_params_['classifier__C'],
        'Best_degree': grid_search.best_params_['classifier__degree'],
        'Best_gamma': grid_search.best_params_['classifier__gamma'],
        'MSE': mse_test,
        'MAE': mae_test,
        'R2': r2_test,
        'Adjusted_R2': adj_r2_test,
    }, index=[0])])

    return scoring_table

In [154]:
param_grid_1 = {
    'classifier__max_iter': [10, 100, 1000, 2000], 
    'classifier__C': [0.01, 0.1, 1.0, 10.0, 100.0],  
    'classifier__degree': [3, 4, 5],
    'classifier__gamma': [0.01, 0.1, 1.0, 10.0]
}

scoring_table = model_evaluation_poly(svr_poly, param_grid_1, scoring_table, 'svr_poly_global', X_train_global, y_train_global, X_test_global, y_test_global)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Librar

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C,Best_degree,Best_gamma
0,svr_poly_global_train,7.206037e+20,9.812925e+09,-0.001423,-0.008012,1000.0,100.0,5.0,0.01
0,svr_poly_global_test,7.924819e+20,1.011201e+10,-0.000463,-0.027328,1000.0,100.0,5.0,0.01


In [155]:
param_grid_1 = {
    'classifier__max_iter': [800, 900, 1000, 1100], 
    'classifier__C': [50.0, 100.0, 150.0, 200.0],  
    'classifier__degree': [3, 4, 5],
    'classifier__gamma': [0.01, 0.02, 0.03]
}

scoring_table = model_evaluation_poly(svr_poly, param_grid_1, scoring_table, 'svr_poly_global_2', X_train_global, y_train_global, X_test_global, y_test_global)
scoring_table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Lib

,Model,MSE,MAE,R2,Adjusted_R2,Best_iteration,Best_C,Best_degree,Best_gamma
0,svr_poly_global_train,7.206037e+20,9.812925e+09,-0.001423,-0.008012,1000.0,100.0,5.0,0.01
0,svr_poly_global_test,7.924819e+20,1.011201e+10,-0.000463,-0.027328,1000.0,100.0,5.0,0.01
0,svr_poly_global_2_train,7.269230e+20,1.116467e+10,-0.010205,-0.016852,900.0,50.0,5.0,0.01
0,svr_poly_global_2_test,7.974227e+20,1.144245e+10,-0.006701,-0.033733,900.0,50.0,5.0,0.01


In conclusion, the unsuitability of SVR for predicting sales forecasts is evident from the poor performance observed in the evaluation metrics, particularly the MSE. The challenges faced by SVR in capturing the complex patterns inherent in the data, along with its sensitivity to hyperparameters and limited interpretability, suggest that alternative models may provide more effective solutions.
